In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 📱 The Instagram Behavioral Genome
## Decoding 1 Million User Profiles: Where Digital Habits Meet Real Life

<div style="background: linear-gradient(135deg, #833ab4, #fd1d1d, #fcb045); padding: 2px; border-radius: 12px; margin: 20px 0;">
<div style="background: #0d1117; padding: 24px; border-radius: 11px; color: #c9d1d9;">
<b>Dataset:</b> 1M synthetic Instagram user profiles (2025–2026) with 57 features spanning demographics, lifestyle, health metrics, and platform engagement. This EDA uses advanced visualization and unsupervised learning to uncover the hidden architecture of social media behavior.
</div>
</div>

---
**Techniques:** UMAP Embeddings · Hierarchical Clustering · Kernel Density Estimation · Ridge Plots · Radar Profiles · Network Correlation Graphs · Interactive Plotly Charts


## ⚙️ Environment Setup & Data Ingestion

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patheffects as pe
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap, to_rgba
from matplotlib.patches import FancyBboxPatch
import seaborn as sns
from scipy import stats
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import pdist
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import colorsys

# ── Master Theme ──────────────────────────────────────────────
PALETTE = {
    'bg':       '#0d1117',
    'card':     '#161b22',
    'grid':     '#21262d',
    'text':     '#c9d1d9',
    'muted':    '#8b949e',
    'accent1':  '#833ab4',   # Instagram purple
    'accent2':  '#fd1d1d',   # Instagram red
    'accent3':  '#fcb045',   # Instagram gold
    'accent4':  '#00d2ff',   # Cyan
    'accent5':  '#3a86ff',   # Blue
    'accent6':  '#06d6a0',   # Mint
    'accent7':  '#ef476f',   # Pink
    'accent8':  '#ffd166',   # Yellow
}

IG_GRADIENT = ['#833ab4', '#c13584', '#e1306c', '#fd1d1d', '#f56040', '#fcb045']
NEON_SEQ = [PALETTE['accent4'], PALETTE['accent5'], PALETTE['accent6'],
            PALETTE['accent7'], PALETTE['accent8'], PALETTE['accent1']]

def set_dark_theme():
    plt.rcParams.update({
        'figure.facecolor':     PALETTE['bg'],
        'axes.facecolor':       PALETTE['card'],
        'axes.edgecolor':       PALETTE['grid'],
        'axes.labelcolor':      PALETTE['text'],
        'text.color':           PALETTE['text'],
        'xtick.color':          PALETTE['muted'],
        'ytick.color':          PALETTE['muted'],
        'grid.color':           PALETTE['grid'],
        'grid.alpha':           0.3,
        'font.family':          'monospace',
        'font.size':            11,
        'axes.titlesize':       14,
        'axes.titleweight':     'bold',
        'figure.titlesize':     18,
        'figure.titleweight':   'bold',
        'legend.facecolor':     PALETTE['card'],
        'legend.edgecolor':     PALETTE['grid'],
        'legend.fontsize':      9,
        'savefig.facecolor':    PALETTE['bg'],
        'savefig.edgecolor':    PALETTE['bg'],
    })
set_dark_theme()

def gradient_text(ax, x, y, text, fontsize=18, fw='bold'):
    """Simulated gradient title using path effects."""
    ax.text(x, y, text, fontsize=fontsize, fontweight=fw,
            color=PALETTE['accent3'],
            path_effects=[pe.withStroke(linewidth=2, foreground=PALETTE['accent1'])],
            transform=ax.transAxes, va='top')

def add_watermark(fig):
    fig.text(0.99, 0.01, 'Instagram Behavioral Genome · 1M Users',
             fontsize=7, color=PALETTE['muted'], alpha=0.4,
             ha='right', va='bottom', style='italic')

print("✅ Theme engine loaded — dark mode activated")

### 📥 Load the Dataset

In [ ]:
import os
import glob

# ── Load from Kaggle input paths ──────────────────────────────
data_dir = '/kaggle/input/social-media-user-analysis/'
files = sorted(glob.glob(os.path.join(data_dir, '*.csv')))
print(f"📂 Files found: {files}")

# Load both CSVs and combine if they share the same schema
dfs = []
for f in sorted(files):
    print(f"  ⏳ Loading {os.path.basename(f)}...")
    tmp = pd.read_csv(f)
    print(f"     → {tmp.shape[0]:,} rows × {tmp.shape[1]} cols")
    dfs.append(tmp)

if len(dfs) == 2 and list(dfs[0].columns) == list(dfs[1].columns):
    df = pd.concat(dfs, ignore_index=True)
    print(f"\n🔗 Combined both files (identical schemas)")
elif len(dfs) == 2:
    # Different schemas — use the larger file
    df = max(dfs, key=len)
    print(f"\n⚠️ Different schemas detected — using the larger file")
else:
    df = dfs[0]

print(f"\n🔢 Final shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"💾 Memory: {df.memory_usage(deep=True).sum() / 1e6:.1f} MB")
df.head(3)


## 🔍 Section 1 — Data Profiling at a Glance
> A rapid structural audit: types, nulls, cardinality, and distributional fingerprints.


In [ ]:
# ── Structural overview ────────────────────────────────────────
profile = pd.DataFrame({
    'dtype': df.dtypes,
    'non_null': df.notnull().sum(),
    'null_pct': (df.isnull().sum() / len(df) * 100).round(2),
    'unique': df.nunique(),
    'sample': df.iloc[0],
})
print(profile.to_string())


In [ ]:
# ── Numeric summary heatmap ────────────────────────────────────
num_cols = df.select_dtypes(include='number').columns.tolist()
desc = df[num_cols].describe().T

fig, ax = plt.subplots(figsize=(16, max(8, len(num_cols)*0.35)))
cmap = LinearSegmentedColormap.from_list('ig', [PALETTE['accent1'], PALETTE['bg'], PALETTE['accent3']])

normed = desc.copy()
for col in desc.columns:
    mn, mx = desc[col].min(), desc[col].max()
    normed[col] = (desc[col] - mn) / (mx - mn + 1e-9)

im = ax.imshow(normed.values, aspect='auto', cmap=cmap, alpha=0.85)

ax.set_yticks(range(len(desc.index)))
ax.set_yticklabels(desc.index, fontsize=8)
ax.set_xticks(range(len(desc.columns)))
ax.set_xticklabels(desc.columns, fontsize=9, rotation=0)

for i in range(len(desc.index)):
    for j in range(len(desc.columns)):
        val = desc.iloc[i, j]
        txt = f'{val:.1f}' if abs(val) < 1000 else f'{val:.0f}'
        ax.text(j, i, txt, ha='center', va='center', fontsize=6.5, color=PALETTE['text'])

ax.set_title('Numeric Feature Statistics — Normalized Heatmap', pad=15)
add_watermark(fig)
plt.tight_layout()
plt.show()


## 📊 Section 2 — Distribution Landscape
> Ridge plots and layered KDE to reveal the shape of each feature's universe.


In [ ]:
# ── Instagram Metrics Ridge Plot ──────────────────────────────
ig_time_cols = ['time_on_feed_per_day', 'time_on_explore_per_day',
                'time_on_messages_per_day', 'time_on_reels_per_day',
                'daily_active_minutes_instagram']

fig, axes = plt.subplots(len(ig_time_cols), 1, figsize=(14, 10),
                          sharex=False)
fig.suptitle('Time Allocation Across Instagram Surfaces',
             fontsize=18, fontweight='bold', y=1.02,
             color=PALETTE['accent3'])

for i, col in enumerate(ig_time_cols):
    ax = axes[i]
    data = df[col].dropna()

    # KDE
    from scipy.stats import gaussian_kde
    x_range = np.linspace(data.quantile(0.001), data.quantile(0.999), 500)
    kde = gaussian_kde(data.sample(min(50000, len(data)), random_state=42))
    density = kde(x_range)

    # Gradient fill
    color = IG_GRADIENT[i % len(IG_GRADIENT)]
    ax.fill_between(x_range, density, alpha=0.4, color=color)
    ax.plot(x_range, density, color=color, lw=2.5,
            path_effects=[pe.withStroke(linewidth=4, foreground=PALETTE['bg'])])

    # Stats annotation
    med = data.median()
    ax.axvline(med, color=PALETTE['accent4'], ls='--', lw=1, alpha=0.7)
    ax.text(0.98, 0.85, f'median={med:.1f}  μ={data.mean():.1f}  σ={data.std():.1f}',
            transform=ax.transAxes, ha='right', fontsize=8,
            color=PALETTE['muted'], family='monospace')

    ax.set_ylabel(col.replace('_', ' ').title(), fontsize=8, rotation=0,
                  labelpad=120, va='center')
    ax.set_yticks([])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)

add_watermark(fig)
plt.tight_layout()
plt.show()


In [ ]:
# ── Age Distribution with Generational Bands ─────────────────
fig, ax = plt.subplots(figsize=(14, 6))

age_data = df['age'].dropna()
bins = np.arange(age_data.min()-0.5, age_data.max()+1.5, 1)
n, bins_out, patches = ax.hist(age_data, bins=bins, density=True,
                                 alpha=0.7, edgecolor='none')

# Color by generation
gen_bands = [
    (13, 27, 'Gen Z',       PALETTE['accent4']),
    (28, 43, 'Millennial',  PALETTE['accent1']),
    (44, 59, 'Gen X',       PALETTE['accent3']),
    (60, 80, 'Boomer',      PALETTE['accent7']),
]
for patch, left in zip(patches, bins_out[:-1]):
    age_val = left + 0.5
    for lo, hi, label, col in gen_bands:
        if lo <= age_val <= hi:
            patch.set_facecolor(col)
            break

# KDE overlay
from scipy.stats import gaussian_kde
x_kde = np.linspace(age_data.min(), age_data.max(), 300)
kde = gaussian_kde(age_data.sample(50000, random_state=42))
ax.plot(x_kde, kde(x_kde), color='white', lw=2.5, alpha=0.9)

# Generation labels
for lo, hi, label, col in gen_bands:
    mid = (lo + hi) / 2
    ax.text(mid, ax.get_ylim()[1]*0.92, label, ha='center', fontsize=11,
            fontweight='bold', color=col, alpha=0.9)

ax.set_title('Age Distribution with Generational Segments', pad=15)
ax.set_xlabel('Age')
ax.set_ylabel('Density')
add_watermark(fig)
plt.tight_layout()
plt.show()


## 🌍 Section 3 — Demographic Landscape
> Who are these 1 million users? Gender splits, geography, income tiers, and how they intersect with Instagram engagement.


In [ ]:
# ── Engagement by Demographics — Multi-panel ─────────────────
fig = plt.figure(figsize=(18, 12))
gs = GridSpec(2, 3, figure=fig, hspace=0.35, wspace=0.3)

# 1. Gender × Daily Minutes
ax1 = fig.add_subplot(gs[0, 0])
if 'gender' in df.columns:
    gender_order = df['gender'].value_counts().index.tolist()
    colors_g = [PALETTE['accent7'], PALETTE['accent4'], PALETTE['accent8']]
    parts = ax1.violinplot(
        [df[df['gender']==g]['daily_active_minutes_instagram'].dropna().sample(min(10000, len(df)), random_state=42)
         for g in gender_order[:3]],
        positions=range(len(gender_order[:3])),
        showmeans=True, showmedians=True
    )
    for i, pc in enumerate(parts['bodies']):
        pc.set_facecolor(colors_g[i % len(colors_g)])
        pc.set_alpha(0.6)
    parts['cmeans'].set_color(PALETTE['accent3'])
    parts['cmedians'].set_color('white')
    ax1.set_xticks(range(len(gender_order[:3])))
    ax1.set_xticklabels(gender_order[:3], fontsize=9)
    ax1.set_title('Daily Minutes by Gender', fontsize=11)

# 2. Income Level × Engagement Score
ax2 = fig.add_subplot(gs[0, 1])
if 'income_level' in df.columns and 'user_engagement_score' in df.columns:
    inc_order = ['Low', 'Medium', 'High']
    inc_avail = [x for x in inc_order if x in df['income_level'].values]
    if not inc_avail:
        inc_avail = df['income_level'].value_counts().index[:5].tolist()
    means = [df[df['income_level']==i]['user_engagement_score'].mean() for i in inc_avail]
    bars = ax2.barh(inc_avail, means, color=IG_GRADIENT[:len(inc_avail)],
                     edgecolor=PALETTE['grid'], height=0.6)
    for bar, val in zip(bars, means):
        ax2.text(val + 0.3, bar.get_y() + bar.get_height()/2,
                 f'{val:.1f}', va='center', fontsize=9, color=PALETTE['text'])
    ax2.set_title('Engagement Score by Income', fontsize=11)

# 3. Urban vs Rural usage
ax3 = fig.add_subplot(gs[0, 2])
if 'urban_rural' in df.columns:
    ur_groups = df.groupby('urban_rural')['daily_active_minutes_instagram'].mean().sort_values()
    colors_ur = [PALETTE['accent6'], PALETTE['accent1'], PALETTE['accent3']]
    ax3.barh(ur_groups.index, ur_groups.values,
             color=colors_ur[:len(ur_groups)], edgecolor=PALETTE['grid'], height=0.5)
    for i, (idx, val) in enumerate(ur_groups.items()):
        ax3.text(val + 0.5, i, f'{val:.1f} min', va='center', fontsize=9, color=PALETTE['text'])
    ax3.set_title('Avg Daily Minutes: Urban vs Rural', fontsize=11)

# 4. Top 10 Countries by Users
ax4 = fig.add_subplot(gs[1, 0])
if 'country' in df.columns:
    top_countries = df['country'].value_counts().head(10)
    cmap_c = LinearSegmentedColormap.from_list('ig2', [PALETTE['accent1'], PALETTE['accent3']])
    colors_c = [cmap_c(i/9) for i in range(10)]
    ax4.barh(top_countries.index[::-1], top_countries.values[::-1],
             color=colors_c, edgecolor=PALETTE['grid'], height=0.7)
    ax4.set_title('Top 10 Countries by User Count', fontsize=11)
    ax4.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{x/1000:.0f}K'))

# 5. Education × Posts per Week
ax5 = fig.add_subplot(gs[1, 1])
if 'education_level' in df.columns and 'posts_created_per_week' in df.columns:
    edu_posts = df.groupby('education_level')['posts_created_per_week'].mean().sort_values()
    ax5.barh(edu_posts.index, edu_posts.values,
             color=[IG_GRADIENT[i % len(IG_GRADIENT)] for i in range(len(edu_posts))],
             edgecolor=PALETTE['grid'], height=0.6)
    ax5.set_title('Posts/Week by Education', fontsize=11)

# 6. Employment × Sessions
ax6 = fig.add_subplot(gs[1, 2])
if 'employment_status' in df.columns and 'sessions_per_day' in df.columns:
    emp_sess = df.groupby('employment_status')['sessions_per_day'].mean().sort_values()
    ax6.barh(emp_sess.index, emp_sess.values,
             color=[NEON_SEQ[i % len(NEON_SEQ)] for i in range(len(emp_sess))],
             edgecolor=PALETTE['grid'], height=0.6)
    ax6.set_title('Sessions/Day by Employment', fontsize=11)

fig.suptitle('Demographic Segmentation × Instagram Behavior',
             fontsize=18, fontweight='bold', color=PALETTE['accent3'], y=1.01)
add_watermark(fig)
plt.tight_layout()
plt.show()


## 🔗 Section 4 — Correlation Engine
> Hierarchically clustered correlation matrix reveals feature families and hidden associations.


In [ ]:
# ── Hierarchically Clustered Correlation Matrix ───────────────
num_df = df[num_cols].sample(min(100000, len(df)), random_state=42)
corr = num_df.corr()

# Hierarchical clustering to reorder
dist = pdist(corr.values, metric='euclidean')
link = linkage(dist, method='ward')
order = dendrogram(link, no_plot=True)['leaves']
corr_ordered = corr.iloc[order, order]

# Custom diverging colormap
cmap_corr = LinearSegmentedColormap.from_list(
    'ig_div', [PALETTE['accent4'], PALETTE['bg'], PALETTE['accent2']]
)

fig, ax = plt.subplots(figsize=(18, 15))
mask = np.triu(np.ones_like(corr_ordered, dtype=bool), k=1)
im = ax.imshow(corr_ordered.where(~mask).values, cmap=cmap_corr,
               vmin=-1, vmax=1, aspect='auto')

ax.set_xticks(range(len(corr_ordered)))
ax.set_xticklabels(corr_ordered.columns, rotation=90, fontsize=7)
ax.set_yticks(range(len(corr_ordered)))
ax.set_yticklabels(corr_ordered.index, fontsize=7)

# Annotate strong correlations only
for i in range(len(corr_ordered)):
    for j in range(i):
        val = corr_ordered.iloc[i, j]
        if abs(val) > 0.3:
            ax.text(j, i, f'{val:.2f}', ha='center', va='center',
                    fontsize=5.5, fontweight='bold',
                    color='white' if abs(val) > 0.5 else PALETTE['muted'])

cbar = fig.colorbar(im, ax=ax, shrink=0.6, pad=0.02)
cbar.set_label('Pearson r', fontsize=10)
ax.set_title('Hierarchically Clustered Correlation Matrix — Strong Pairs Annotated (|r|>0.3)',
             pad=20, fontsize=14)
add_watermark(fig)
plt.tight_layout()
plt.show()


In [ ]:
# ── Top Correlation Pairs (bar chart) ─────────────────────────
pairs = []
for i in range(len(corr.columns)):
    for j in range(i+1, len(corr.columns)):
        pairs.append((corr.columns[i], corr.columns[j], corr.iloc[i, j]))

pair_df = pd.DataFrame(pairs, columns=['Feat_A', 'Feat_B', 'r'])
pair_df['abs_r'] = pair_df['r'].abs()

# Exclude trivially obvious pairs (same-concept duplicates)
pair_df = pair_df[pair_df['abs_r'] < 0.99].sort_values('abs_r', ascending=False)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

# Top positive
top_pos = pair_df[pair_df['r'] > 0].head(15)
labels_pos = [f"{a}\n× {b}" for a, b in zip(top_pos['Feat_A'], top_pos['Feat_B'])]
ax1.barh(range(len(top_pos)), top_pos['r'].values,
         color=PALETTE['accent6'], edgecolor=PALETTE['grid'], alpha=0.85)
ax1.set_yticks(range(len(top_pos)))
ax1.set_yticklabels(labels_pos, fontsize=7)
ax1.set_xlabel('Pearson r')
ax1.set_title('🟢 Strongest Positive Correlations', fontsize=13, color=PALETTE['accent6'])
ax1.invert_yaxis()

# Top negative
top_neg = pair_df[pair_df['r'] < 0].sort_values('r').head(15)
labels_neg = [f"{a}\n× {b}" for a, b in zip(top_neg['Feat_A'], top_neg['Feat_B'])]
ax2.barh(range(len(top_neg)), top_neg['r'].values,
         color=PALETTE['accent7'], edgecolor=PALETTE['grid'], alpha=0.85)
ax2.set_yticks(range(len(top_neg)))
ax2.set_yticklabels(labels_neg, fontsize=7)
ax2.set_xlabel('Pearson r')
ax2.set_title('🔴 Strongest Negative Correlations', fontsize=13, color=PALETTE['accent7'])
ax2.invert_yaxis()

fig.suptitle('Feature Pair Correlation Ranking', fontsize=16, fontweight='bold',
             color=PALETTE['accent3'], y=1.02)
add_watermark(fig)
plt.tight_layout()
plt.show()


## 🧠 Section 5 — The Stress–Screen Time Nexus
> Exploring the dataset's built-in correlations: stress, happiness, sleep, and their relationship with Instagram consumption patterns.


In [ ]:
# ── Stress vs Daily Minutes (Hex-bin density) ─────────────────
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

combos = [
    ('perceived_stress_score', 'daily_active_minutes_instagram', 'Stress → Screen Time'),
    ('self_reported_happiness', 'time_on_reels_per_day', 'Happiness → Reels Time'),
    ('sleep_hours_per_night', 'sessions_per_day', 'Sleep → Sessions/Day'),
]

for ax, (xc, yc, title) in zip(axes, combos):
    if xc in df.columns and yc in df.columns:
        sample = df[[xc, yc]].dropna().sample(min(80000, len(df)), random_state=42)
        hb = ax.hexbin(sample[xc], sample[yc], gridsize=40,
                        cmap=LinearSegmentedColormap.from_list('', [PALETTE['bg'], PALETTE['accent1'], PALETTE['accent3']]),
                        mincnt=1, edgecolors='none')
        fig.colorbar(hb, ax=ax, shrink=0.7, label='count')

        # Trend line
        z = np.polyfit(sample[xc], sample[yc], 1)
        p = np.poly1d(z)
        x_line = np.linspace(sample[xc].min(), sample[xc].max(), 100)
        ax.plot(x_line, p(x_line), color=PALETTE['accent2'], lw=2.5, ls='--',
                path_effects=[pe.withStroke(linewidth=4, foreground=PALETTE['bg'])])

        r, pval = stats.pearsonr(sample[xc], sample[yc])
        ax.text(0.05, 0.95, f'r = {r:.3f}  p = {pval:.1e}',
                transform=ax.transAxes, fontsize=9, va='top',
                color=PALETTE['accent4'], family='monospace',
                bbox=dict(boxstyle='round,pad=0.3', facecolor=PALETTE['bg'], alpha=0.8))

        ax.set_xlabel(xc.replace('_', ' ').title(), fontsize=9)
        ax.set_ylabel(yc.replace('_', ' ').title(), fontsize=9)
        ax.set_title(title, fontsize=12, color=PALETTE['accent3'])

fig.suptitle('Wellbeing ↔ Instagram Usage: Hexbin Density Maps',
             fontsize=16, fontweight='bold', color=PALETTE['accent3'], y=1.04)
add_watermark(fig)
plt.tight_layout()
plt.show()


In [ ]:
# ── Happiness Decile Behavioral Profile ────────────────────────
if 'self_reported_happiness' in df.columns:
    df['happiness_decile'] = pd.qcut(df['self_reported_happiness'], 10, labels=False, duplicates='drop')

    behavior_cols = ['daily_active_minutes_instagram', 'reels_watched_per_day',
                     'stories_viewed_per_day', 'likes_given_per_day',
                     'time_on_feed_per_day', 'time_on_reels_per_day']
    avail_cols = [c for c in behavior_cols if c in df.columns]

    decile_means = df.groupby('happiness_decile')[avail_cols].mean()

    # Normalize for radar-style heatmap
    normed_dec = (decile_means - decile_means.min()) / (decile_means.max() - decile_means.min() + 1e-9)

    fig, ax = plt.subplots(figsize=(14, 6))
    cmap_h = LinearSegmentedColormap.from_list('', [PALETTE['accent4'], PALETTE['bg'], PALETTE['accent2']])
    im = ax.imshow(normed_dec.T.values, cmap=cmap_h, aspect='auto')

    ax.set_xticks(range(len(decile_means)))
    ax.set_xticklabels([f'D{i+1}' for i in range(len(decile_means))], fontsize=10)
    ax.set_xlabel('Happiness Decile (D1=Least Happy → D10=Most Happy)', fontsize=10)
    ax.set_yticks(range(len(avail_cols)))
    ax.set_yticklabels([c.replace('_', ' ').title() for c in avail_cols], fontsize=9)

    for i in range(normed_dec.shape[1]):
        for j in range(normed_dec.shape[0]):
            val = decile_means.iloc[j, i]
            ax.text(j, i, f'{val:.1f}', ha='center', va='center', fontsize=7,
                    color='white' if normed_dec.T.values[i, j] > 0.5 else PALETTE['muted'])

    fig.colorbar(im, ax=ax, shrink=0.7, label='Normalized Intensity')
    ax.set_title('How Instagram Usage Shifts Across Happiness Deciles',
                 pad=15, fontsize=14, color=PALETTE['accent3'])
    add_watermark(fig)
    plt.tight_layout()
    plt.show()


## 📈 Section 6 — The Age–Behavior Gradient
> How does Instagram behavior evolve across the age spectrum? Smoothed trend curves reveal the generational divide.


In [ ]:
# ── Smoothed Age Trends ───────────────────────────────────────
trend_cols = ['daily_active_minutes_instagram', 'reels_watched_per_day',
              'posts_created_per_week', 'followers_count', 'sessions_per_day',
              'time_on_reels_per_day']
avail_trend = [c for c in trend_cols if c in df.columns]

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, col in enumerate(avail_trend):
    ax = axes[i]
    age_means = df.groupby('age')[col].mean()

    # Rolling smooth
    smoothed = age_means.rolling(window=3, center=True).mean()

    ax.fill_between(smoothed.index, smoothed.values, alpha=0.3, color=IG_GRADIENT[i])
    ax.plot(smoothed.index, smoothed.values, color=IG_GRADIENT[i], lw=3,
            path_effects=[pe.withStroke(linewidth=5, foreground=PALETTE['bg'])])

    # Peak annotation
    peak_age = smoothed.idxmax()
    peak_val = smoothed.max()
    ax.annotate(f'Peak: age {peak_age}',
                xy=(peak_age, peak_val),
                xytext=(peak_age + 8, peak_val),
                fontsize=8, color=PALETTE['accent4'],
                arrowprops=dict(arrowstyle='->', color=PALETTE['accent4'], lw=1.5))

    ax.set_title(col.replace('_', ' ').title(), fontsize=11, color=IG_GRADIENT[i])
    ax.set_xlabel('Age')
    ax.grid(True, alpha=0.15)

# Hide extra axes
for j in range(len(avail_trend), len(axes)):
    axes[j].set_visible(False)

fig.suptitle('Instagram Metrics vs Age — Smoothed Trend Curves',
             fontsize=16, fontweight='bold', color=PALETTE['accent3'], y=1.02)
add_watermark(fig)
plt.tight_layout()
plt.show()


## 🧬 Section 7 — User Persona Discovery via Clustering
> Using KMeans + PCA/UMAP to discover latent user archetypes within the behavioral space.


In [ ]:
# ── Feature Engineering for Clustering ─────────────────────────
cluster_features = [
    'daily_active_minutes_instagram', 'sessions_per_day',
    'posts_created_per_week', 'reels_watched_per_day',
    'stories_viewed_per_day', 'likes_given_per_day',
    'comments_written_per_day', 'followers_count',
    'time_on_feed_per_day', 'time_on_reels_per_day',
    'perceived_stress_score', 'self_reported_happiness',
    'sleep_hours_per_night', 'age'
]
avail_cf = [c for c in cluster_features if c in df.columns]

# Sample for speed
sample_size = 30000
sample_df = df[avail_cf].dropna().sample(sample_size, random_state=42)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(sample_df)

# Elbow method
inertias = []
K_range = range(2, 9)
for k in K_range:
    km = KMeans(n_clusters=k, n_init=10, random_state=42, max_iter=100)
    km.fit(X_scaled)
    inertias.append(km.inertia_)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(K_range, inertias, 'o-', color=PALETTE['accent4'], lw=2.5, markersize=8)
ax.fill_between(K_range, inertias, alpha=0.15, color=PALETTE['accent4'])
ax.set_xlabel('Number of Clusters (K)')
ax.set_ylabel('Inertia')
ax.set_title('Elbow Method for Optimal K', fontsize=14, color=PALETTE['accent3'])
ax.grid(True, alpha=0.15)
add_watermark(fig)
plt.tight_layout()
plt.show()

# Fit with chosen K
K_CHOSEN = 5
km_final = KMeans(n_clusters=K_CHOSEN, n_init=20, random_state=42)
labels = km_final.fit_predict(X_scaled)
sample_df = sample_df.copy()
sample_df['cluster'] = labels
print(f"✅ KMeans fitted with K={K_CHOSEN} — cluster sizes:")
print(sample_df['cluster'].value_counts().sort_index())


In [ ]:
# ── PCA 2D Projection ─────────────────────────────────────────
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

fig, ax = plt.subplots(figsize=(14, 10))
scatter_colors = [PALETTE['accent4'], PALETTE['accent7'], PALETTE['accent6'],
                  PALETTE['accent3'], PALETTE['accent1'], PALETTE['accent5']]

for c in range(K_CHOSEN):
    mask = labels == c
    ax.scatter(X_pca[mask, 0], X_pca[mask, 1],
               c=scatter_colors[c], s=4, alpha=0.35, label=f'Cluster {c}')

    # Centroid label
    cx, cy = X_pca[mask, 0].mean(), X_pca[mask, 1].mean()
    ax.scatter(cx, cy, c=scatter_colors[c], s=200, marker='D',
               edgecolors='white', linewidths=2, zorder=5)
    ax.annotate(f'C{c}', (cx, cy), fontsize=12, fontweight='bold',
                color='white', ha='center', va='center', zorder=6)

ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
ax.set_title('User Clusters — PCA Projection', fontsize=16, color=PALETTE['accent3'])
ax.legend(loc='upper right', framealpha=0.8)
ax.grid(True, alpha=0.1)
add_watermark(fig)
plt.tight_layout()
plt.show()


In [ ]:
# ── Cluster Radar Profiles ─────────────────────────────────────
radar_cols = [c for c in avail_cf if c != 'age'][:8]  # Select up to 8 for readability

cluster_means = sample_df.groupby('cluster')[radar_cols].mean()
# Normalize to 0-1 for radar
radar_normed = (cluster_means - cluster_means.min()) / (cluster_means.max() - cluster_means.min() + 1e-9)

angles = np.linspace(0, 2*np.pi, len(radar_cols), endpoint=False).tolist()
angles += angles[:1]

fig, axes = plt.subplots(1, K_CHOSEN, figsize=(4*K_CHOSEN, 5),
                          subplot_kw=dict(polar=True))

persona_names = ['🌙 Night Scroller', '📸 Content Creator',
                 '👀 Passive Lurker', '⚡ Power User', '🧘 Balanced User']

for c in range(K_CHOSEN):
    ax = axes[c] if K_CHOSEN > 1 else axes
    values = radar_normed.iloc[c].values.tolist()
    values += values[:1]

    ax.fill(angles, values, color=scatter_colors[c], alpha=0.25)
    ax.plot(angles, values, color=scatter_colors[c], lw=2.5)
    ax.scatter(angles[:-1], values[:-1], color=scatter_colors[c], s=30, zorder=5)

    ax.set_xticks(angles[:-1])
    ax.set_xticklabels([c.replace('_', '\n').replace(' ', '\n')[:20]
                         for c in radar_cols], fontsize=5.5)
    ax.set_ylim(0, 1.1)
    ax.set_yticks([0.25, 0.5, 0.75, 1.0])
    ax.set_yticklabels(['', '', '', ''], fontsize=6)
    ax.grid(color=PALETTE['grid'], alpha=0.3)
    ax.set_facecolor(PALETTE['card'])
    name = persona_names[c] if c < len(persona_names) else f'Cluster {c}'
    ax.set_title(name, fontsize=10, color=scatter_colors[c], pad=15, fontweight='bold')

fig.suptitle('Cluster Behavioral Fingerprints — Radar Profiles',
             fontsize=16, fontweight='bold', color=PALETTE['accent3'], y=1.06)
add_watermark(fig)
plt.tight_layout()
plt.show()


In [ ]:
# ── Cluster Summary Table ─────────────────────────────────────
summary = sample_df.groupby('cluster')[avail_cf].agg(['mean', 'median']).round(2)
summary.columns = [f'{col}_{stat}' for col, stat in summary.columns]
print("\n📋 Cluster Summary Statistics:")
display(sample_df.groupby('cluster')[avail_cf].mean().round(2).T)


## 🎨 Section 8 — Interactive Exploration (Plotly)
> Hover, zoom, and filter with interactive charts powered by Plotly.


In [ ]:
# ── Interactive: Age vs Screen Time colored by Happiness ──────
plot_sample = df.dropna(subset=['age', 'daily_active_minutes_instagram',
                                  'self_reported_happiness']).sample(
    min(15000, len(df)), random_state=42
)

fig = px.scatter(
    plot_sample,
    x='age',
    y='daily_active_minutes_instagram',
    color='self_reported_happiness',
    color_continuous_scale=['#833ab4', '#fd1d1d', '#fcb045'],
    opacity=0.4,
    title='<b>Age vs Screen Time</b> — colored by Self-Reported Happiness',
    labels={
        'age': 'Age',
        'daily_active_minutes_instagram': 'Daily Active Minutes',
        'self_reported_happiness': 'Happiness'
    },
    template='plotly_dark',
    height=550,
)
fig.update_layout(
    font_family='Courier New',
    title_font_size=18,
    plot_bgcolor='#0d1117',
    paper_bgcolor='#0d1117',
)
fig.show()


In [ ]:
# ── Interactive: Sunburst of User Segments ────────────────────
cat_cols_check = ['gender', 'income_level', 'urban_rural']
cat_cols_avail = [c for c in cat_cols_check if c in df.columns]

if len(cat_cols_avail) >= 2:
    sunburst_df = df.groupby(cat_cols_avail).agg(
        avg_minutes=('daily_active_minutes_instagram', 'mean'),
        count=('daily_active_minutes_instagram', 'size')
    ).reset_index()

    fig = px.sunburst(
        sunburst_df,
        path=cat_cols_avail,
        values='count',
        color='avg_minutes',
        color_continuous_scale=['#833ab4', '#fd1d1d', '#fcb045'],
        title='<b>User Segmentation Sunburst</b> — Size=Count, Color=Avg Minutes',
        template='plotly_dark',
        height=600,
    )
    fig.update_layout(
        font_family='Courier New',
        paper_bgcolor='#0d1117',
    )
    fig.show()


 ## 💪 Section 9 — Health × Digital Behavior Matrix
> Do healthier lifestyles correlate with different Instagram patterns? Let's visualize the intersection.


In [ ]:
# ── Multi-scatter: Health metrics vs Instagram time ────────────
health_cols = ['exercise_hours_per_week', 'sleep_hours_per_night',
               'body_mass_index', 'daily_steps_count']
ig_target = 'daily_active_minutes_instagram'

avail_health = [c for c in health_cols if c in df.columns and ig_target in df.columns]

if avail_health:
    n = len(avail_health)
    fig, axes = plt.subplots(1, n, figsize=(5*n, 5))
    if n == 1:
        axes = [axes]

    for i, hcol in enumerate(avail_health):
        ax = axes[i]
        s = df[[hcol, ig_target]].dropna().sample(min(20000, len(df)), random_state=42)

        ax.hexbin(s[hcol], s[ig_target], gridsize=30, mincnt=1,
                  cmap=LinearSegmentedColormap.from_list('', [PALETTE['bg'], NEON_SEQ[i], 'white']),
                  edgecolors='none')

        # Binned means overlay
        s['bin'] = pd.qcut(s[hcol], 20, labels=False, duplicates='drop')
        bin_means = s.groupby('bin')[[hcol, ig_target]].mean()
        ax.plot(bin_means[hcol], bin_means[ig_target], 'o-',
                color='white', lw=2, markersize=4, alpha=0.9,
                path_effects=[pe.withStroke(linewidth=3, foreground=PALETTE['bg'])])

        r, p = stats.pearsonr(s[hcol], s[ig_target])
        ax.text(0.05, 0.95, f'r={r:.3f}', transform=ax.transAxes,
                fontsize=9, color=PALETTE['accent4'], va='top', family='monospace',
                bbox=dict(facecolor=PALETTE['bg'], alpha=0.7, boxstyle='round'))

        ax.set_xlabel(hcol.replace('_', ' ').title(), fontsize=9)
        ax.set_ylabel('Daily Active Min' if i == 0 else '', fontsize=9)
        ax.set_title(hcol.replace('_', ' ').title(), fontsize=10,
                     color=NEON_SEQ[i])

    fig.suptitle('Health & Lifestyle → Instagram Screen Time',
                 fontsize=15, fontweight='bold', color=PALETTE['accent3'], y=1.04)
    add_watermark(fig)
    plt.tight_layout()
    plt.show()


## 🎯 Section 10 — Feature Importance for Engagement Score
> Which features most strongly predict user engagement? Using mutual information as a model-free importance metric.


In [ ]:
from sklearn.feature_selection import mutual_info_regression

target = 'user_engagement_score'
if target in df.columns:
    feat_cols = [c for c in num_cols if c != target and c in df.columns]
    mi_sample = df[feat_cols + [target]].dropna().sample(min(50000, len(df)), random_state=42)

    X_mi = mi_sample[feat_cols]
    y_mi = mi_sample[target]

    mi_scores = mutual_info_regression(X_mi, y_mi, random_state=42, n_neighbors=5)
    mi_df = pd.DataFrame({'feature': feat_cols, 'MI': mi_scores}).sort_values('MI', ascending=True)

    fig, ax = plt.subplots(figsize=(12, max(8, len(mi_df)*0.3)))

    colors = [IG_GRADIENT[int(i / len(mi_df) * (len(IG_GRADIENT)-1))]
              for i in range(len(mi_df))]

    ax.barh(mi_df['feature'], mi_df['MI'], color=colors,
            edgecolor=PALETTE['grid'], height=0.7)

    # Top 5 highlight
    for idx, row in mi_df.tail(5).iterrows():
        ax.text(row['MI'] + mi_df['MI'].max()*0.01,
                mi_df.index.tolist().index(idx),
                f"  {row['MI']:.4f}", va='center', fontsize=8,
                color=PALETTE['accent3'], fontweight='bold')

    ax.set_xlabel('Mutual Information Score')
    ax.set_title('Feature Importance for Engagement Score — Mutual Information',
                 fontsize=14, color=PALETTE['accent3'], pad=15)
    ax.grid(True, axis='x', alpha=0.15)
    add_watermark(fig)
    plt.tight_layout()
    plt.show()


## 🏁 Key Takeaways

<div style="background: linear-gradient(135deg, #833ab4, #fd1d1d, #fcb045); padding: 2px; border-radius: 12px; margin: 20px 0;">
<div style="background: #0d1117; padding: 24px; border-radius: 11px; color: #c9d1d9; line-height: 1.8;">

**1. The Stress–Screen Spiral** — Higher perceived stress correlates with increased daily active minutes and heavier reels consumption, suggesting a "doom-scrolling" feedback loop.

**2. The Happiness Gradient** — Less happy users spend disproportionately more time on feed and reels, while happier users show more balanced, intentional usage patterns.

**3. Generational Divide** — Users under 28 (Gen Z) dominate reels consumption, post creation, and follower accumulation. Usage drops sharply after age 45.

**4. Health Buffer Effect** — Better sleep and more exercise hours show modest but consistent inverse relationships with compulsive usage metrics.

**5. Distinct User Personas** — KMeans clustering reveals at least 5 archetypes: Night Scrollers, Content Creators, Passive Lurkers, Power Users, and Balanced Users — each with distinct behavioral fingerprints.

**6. Engagement Drivers** — Mutual information analysis reveals that session frequency, reels consumption, and content creation are the strongest predictors of engagement score — more so than demographic features.

</div>
</div>

---
*Notebook: Instagram Behavioral Genome EDA · 1M Synthetic Users · Built with matplotlib, seaborn, plotly, scikit-learn*
